In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import warnings
from sklearn.model_selection import train_test_split
import xgboost as xgb

warnings.filterwarnings("ignore")

# 读取数据
data = pd.read_excel('HV.xlsx')

# 分割数据
X = data.iloc[:, 1:]
Y = data.iloc[:, 0]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=75)

# 数据归一化
scaler = MinMaxScaler()
x_train_normalized = scaler.fit_transform(x_train)

# 确保特征维度
print(f"x_train shape: {x_train.shape}")  # 输出训练集特征的形状

# 最佳参数
best_params = {
    'colsample_bytree': 0.7537221430165258,
    'learning_rate': 0.07141712725507017,
    'max_depth': 3,
    'min_child_weight': 1.036994365622379,
    'n_estimators': 179,
    'reg_alpha': 0.07844930740270828,
    'reg_lambda': 0.4659923785137676,
    'subsample': 0.5124079817369307,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

# 使用最佳参数训练最终模型
xgb_model = xgb.XGBRegressor(**best_params)
xgb_model.fit(x_train, y_train)

# 计算样本集中值
def compute_central_location(X):
    return np.mean(X, axis=0)

# 计算左右偏度
def compute_skewness(X, CL):
    N_L = np.sum(X < CL, axis=0)
    N_U = np.sum(X > CL, axis=0)
    skewness_L = N_L / (N_L + N_U + 1)
    skewness_U = N_U / (N_L + N_U + 1)
    return skewness_L, skewness_U

# 计算不对称接受域范围
def compute_asymmetric_range(X, CL, skewness_L, skewness_U):
    LB = CL - (1 / skewness_U) * (CL - np.min(X, axis=0))
    UB = CL + (1 / skewness_L) * (np.max(X, axis=0) - CL)
    return LB, UB

# PSO算法生成虚拟输入特征
def generate_virtual_samples(X, num_samples=100, num_particles=30, num_iterations=100):
    CL = compute_central_location(X)
    skewness_L, skewness_U = compute_skewness(X, CL)
    LB, UB = compute_asymmetric_range(X, CL, skewness_L, skewness_U)
    
    all_samples = []
    
    while len(all_samples) < num_samples:
        particles = np.random.uniform(LB, UB, (num_particles, X.shape[1]))
        velocities = np.zeros_like(particles)
        personal_best = particles.copy()
        global_best = particles[np.argmin(np.std(particles, axis=1))]

        for _ in range(num_iterations):
            r1, r2 = np.random.rand(), np.random.rand()
            velocities = 0.7 * velocities + 1.5 * r1 * (personal_best - particles) + 1.5 * r2 * (global_best - particles)
            particles = particles + velocities
            particles = np.clip(particles, LB, UB)
            particles = np.clip(particles, 0, None)  # 确保所有特征值为正
            
            current_best = particles[np.argmin(np.std(particles, axis=1))]
            if np.std(current_best) < np.std(global_best):
                global_best = current_best

        for particle in particles:
            if len(all_samples) < num_samples and not any(np.array_equal(particle, s) for s in all_samples):
                all_samples.append(particle)
    
    virtual_samples = np.array(all_samples)
    print(f"Generated virtual samples shape: {virtual_samples.shape}")  # 调试信息
    return virtual_samples

# 生成虚拟输入特征
virtual_features_normalized = generate_virtual_samples(x_train_normalized, num_samples=200)

# 反归一化处理
virtual_features = scaler.inverse_transform(virtual_features_normalized)

# 确保虚拟输入特征的形状与训练集特征形状一致
assert virtual_features.shape[1] == x_train.shape[1], "虚拟样本的特征维度与训练集特征维度不一致！"

# 预测虚拟样本的输出标签
virtual_labels = xgb_model.predict(virtual_features)

# 应用Z-Score异常值检测
z_scores = np.abs(stats.zscore(virtual_features, axis=0))
outliers = (z_scores > 3).any(axis=1)
virtual_features_filtered = virtual_features[~outliers]
virtual_labels_filtered = virtual_labels[~outliers]

# 打印生成的虚拟样本
print("生成的虚拟样本（输入特征）：\n", virtual_features_filtered)
print("生成的虚拟样本（输出标签）：\n", virtual_labels_filtered)

# 将虚拟样本数据导出到Excel
virtual_data = pd.DataFrame(virtual_features_filtered, columns=X.columns)
virtual_data['Predicted_Output'] = virtual_labels_filtered
virtual_data.to_excel('HB_virtual_samples_filtered200.xlsx', index=False)
print("虚拟样本数据已导出到 virtual_samples_filtered.xlsx")


x_train shape: (128, 8)
Generated virtual samples shape: (200, 8)
生成的虚拟样本（输入特征）：
 [[15.57622785  1.3816608   1.68064385 ...  0.45845426  0.15625901
   0.07690557]
 [ 2.3208763   1.81545267  1.0125615  ...  0.          0.35773242
   0.16869524]
 [17.55619739  3.59338144  1.30702722 ...  0.          0.
   0.08344519]
 ...
 [20.47967344  2.14006664  1.65962047 ...  0.47240079  0.15694444
   0.09636004]
 [16.93895779  2.32448462  1.61012774 ...  0.48833434  0.20644916
   0.11351823]
 [18.46960727  2.3051777   1.61308923 ...  0.58433077  0.16746284
   0.10885379]]
生成的虚拟样本（输出标签）：
 [149.3577   122.115685 146.26369   98.305115  79.73907  166.88121
 126.76291  102.8683   133.83429  105.92302  134.49457  100.99107
 100.83794  127.58058  191.55267  105.11311  130.7635   121.963715
 140.05945   69.91075  100.18631   91.572845 177.8845   173.55025
 103.36949  132.20218  141.43962  132.64381   84.66054  128.9072
  57.609306  57.609306  73.64719   81.3534    48.824524  60.584694
  56.960136  43.80632

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

# 读取数据
data = pd.read_excel('HV_virtual_samples_filtered200.xlsx')

# 数据归一化
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

# 将归一化数据转换为DataFrame
data_normalized_df = pd.DataFrame(data_normalized, columns=data.columns)

# 将归一化数据导出为Excel文件
data_normalized_df.to_excel('HV_kuo_normalized200.xlsx', index=False)

print("Normalized data has been saved to 'HV_normalized.xlsx'.")


Normalized data has been saved to 'UTS_normalized.xlsx'.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb
import shap

# 读取数据
data = pd.read_excel('HV.xlsx')

# 分割数据
X = data.iloc[:, 1:]
Y = data.iloc[:, 0]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=75)

# 最佳参数
best_params = {
    'colsample_bytree': 0.7537221430165258,
    'learning_rate': 0.07141712725507017,
    'max_depth': 3,
    'min_child_weight': 1.036994365622379,
    'n_estimators': 179,
    'reg_alpha': 0.07844930740270828,
    'reg_lambda': 0.4659923785137676,
    'subsample': 0.5124079817369307,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

# 使用最佳参数训练最终模型
xgb_model = xgb.XGBRegressor(**best_params)
xgb_model.fit(x_train, y_train)


# 计算SHAP值
explainer = shap.Explainer(xgb_model)
shap_values = explainer(x_test)

# 初始化JavaScript库
shap.initjs()

# 特征重要性排序柱状图
shap.plots.bar(shap_values)

# 使用Matplotlib绘制SHAP力图，手动增加显示特征
plt.figure()
shap_values_sample = shap_values[0]  # 选择第一个样本
feature_importance = np.abs(shap_values_sample.values).argsort()[::-1]  # 按照重要性排序特征
top_features = feature_importance[:10]  # 选择最重要的10个特征

shap.force_plot(explainer.expected_value, shap_values_sample.values[top_features], x_test.iloc[0, top_features], matplotlib=True)
plt.show()

# SHAP值summary plot
shap.summary_plot(shap_values, x_test, plot_type="bar")

# SHAP summary plot with distribution
shap.summary_plot(shap_values, x_test)